In [1]:
import ollama
import gradio as gr
from t2 import get_events_info_from_city
import json
import nbimporter
from city_corrections import get_city

In [2]:
MODEL = "llama3.2"

In [3]:
system_prompt = "You are EventAIde, an efficient AI assistant that helps users find events in the city the user mentioned. "
system_prompt += "At the start of every reply, introduce yourself as 'EventAIde'. "
system_prompt += "Always ask user for the city and if the user doesnt mention then give any city's events. "
system_prompt += "Give only the requested event details in a short, friendly, and professional reply. "
system_prompt += "If you do not KNOW the requested event details for a city, simply reply:\
'Sorry, I do not have information for that city at this time.' \
Never make up any details or invent events."

system_message = [{"role":"system", "content": system_prompt}]

In [4]:
event_function = {
    "name": "get_events_info_from_city",
    "description": "Get the events happening from the given city. Call this whenever you need to know the events list, for example when a customer asks ‘what events are there in New York'",
    "parameters": {
        "type": "object",
        "properties": {
            "city": {
                "type": "string",
                "description": "All the available events in the customer provided city",
            },
        },
        "required": ["city"],
        "additionalProperties": False
    }
}
tools = [{"type": "function", "function": event_function}]

In [5]:
def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    arguments = tool_call.function.arguments
    city = get_city(arguments.get('city'))
    events = get_events_info_from_city(city)
    response = {
        "role": "tool",
        "content": json.dumps({"city": city,"events": events})
    }
    return response, city

In [6]:
def chatbot(message, history):
    messages = system_message+history+[{"role":"user", "content":message}]
    response = ollama.chat(model = MODEL, messages=messages, tools=tools)
    if response.message.tool_calls:
        message = response.message
        response, city = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        response = ollama.chat(model=MODEL, messages=messages)
        
    return response["message"]["content"]

In [7]:
gr.ChatInterface(fn = chatbot, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
